In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Evaluation
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)

# Deep learning
import tensorflow as tf
from tensorflow import keras
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard,Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (
    Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Input,
    BatchNormalization, Activation, SeparableConv2D, Reshape, Multiply, Add
)


from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight


# Utilities
import os
import gc
from tqdm import tqdm
import datetime
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
categories = ['Potato___Early_blight','Potato___Late_blight','Potato___healthy']

In [3]:
data = []
labels = []
path = "/kaggle/input/plant-village/PlantVillage"

for index, cat in enumerate(categories):
    cat_path = os.path.join(path, cat)
    
    print(f"category => : {cat} and size is {len(os.listdir(cat_path))} ")
    for img_name in tqdm(os.listdir(cat_path), desc=f"{cat:15}"):
        img_path = os.path.join(cat_path, img_name)
        img = cv2.imread(img_path)

        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # save RGB not BGR
            img = cv2.resize(img, (224, 224))           # Resize for VGG
            data.append(img)
            labels.append(index)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/plant-village/PlantVillage\\Potato___Early_blight'

In [ ]:
X = np.array(data)
y = np.array(labels)

: 

In [ ]:
X_norm = np.array(data, dtype='float32') / 255.0

: 

In [ ]:
print(X.shape)
print(y.shape)

: 

In [ ]:
plt.figure(figsize=(20, 20))

for n, i in enumerate(list(np.random.randint(0, len(X_norm), 18))):
    plt.subplot(6, 6, n + 1)
    plt.imshow(X[i])
    plt.axis('off')
    plt.title(categories[y[i]])  

: 

## Split

In [ ]:
gc.collect()
tf.keras.backend.clear_session()
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, stratify=y, random_state=42)

: 

In [ ]:
def check_data_distribution():
    print("="*50)
    print("📊 Data Distribution Check:")
    print("="*50)
    
    # Ranges
    print(f"\nData Ranges:")
    print(f" X_train: [{X_train.min():.3f}, {X_train.max():.3f}]")
    print(f" X_test: [{X_test.min():.3f}, {X_test.max():.3f}]")
    
    # Sizes
    print(f"\nDataset Sizes:")
    print(f" Train: {len(X_train)} samples")
    print(f" Test: {len(X_test)} samples")
    
    # Class distribution
    print(f"\nClass Distribution:")
    train_classes = np.bincount(y_train)
    test_classes = np.bincount(y_test)
    
    for i in range(len(train_classes)):
        print(f"  Class {i}: Train={train_classes[i]}, Test={test_classes[i]}")
    
    # Check for imbalance
    imbalance_ratio = max(train_classes) / min(train_classes)
    if imbalance_ratio > 2:
        print(f"\nClass imbalance detected! Ratio: {imbalance_ratio:.2f}")
    else:
        print(f"\nClasses are balanced. Ratio: {imbalance_ratio:.2f}")

check_data_distribution()

: 

## convert to on hot encode

In [ ]:
y_train_cat = to_categorical(y_train, num_classes=3)
y_test_cat = to_categorical(y_test, num_classes=3)

print(f"y_train_cat shape: {y_train_cat.shape}")
print(f"y_train_cat sample: {y_train_cat[0]}")

: 

## Calculate Class Weights

In [ ]:
y_train_classes = np.argmax(y_train_cat, axis=1)
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train_classes),
    y=y_train_classes
)
class_weight_dict = dict(enumerate(class_weights))

print("Class Weights:")
print(f" Class 0 (Early Blight): {class_weight_dict[0]:.2f}")
print(f" Class 1 (Late Blight): {class_weight_dict[1]:.2f}")
print(f" Class 2 (Healthy): {class_weight_dict[2]:.2f}")

: 

## Model 1 -> CNN

In [ ]:
def se_block(input_tensor, reduction_ratio=16):
    """
    Squeeze-and-Excitation Block
    Adaptively recalibrates channel-wise feature responses
    """
    channels = input_tensor.shape[-1]
    
    # Squeeze: Global information embedding
    se = GlobalAveragePooling2D()(input_tensor)
    
    # Excitation: Adaptive recalibration
    se = Dense(channels // reduction_ratio, activation='relu',kernel_regularizer=l2(0.0001))(se)
    se = Dense(channels, activation='sigmoid',kernel_regularizer=l2(0.0001))(se)
    
    # Reshape for multiplication
    se = Reshape((1, 1, channels))(se)
    
    # Scale
    return Multiply()([input_tensor, se])

: 

In [ ]:
def residual_block(x, filters, kernel_size=3, stride=1, use_se=True):
    """
    Residual Block with optional SE attention
    """
    shortcut = x
    
    # First conv
    x = SeparableConv2D(
        filters, kernel_size, strides=stride, padding='same',
        depthwise_initializer='he_uniform',
        pointwise_initializer='he_uniform',
        depthwise_regularizer=l2(0.0001),
        pointwise_regularizer=l2(0.0001)
    )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Second conv
    x = SeparableConv2D(
        filters, kernel_size, padding='same',
        depthwise_initializer='he_uniform',
        pointwise_initializer='he_uniform',
        depthwise_regularizer=l2(0.0001),
        pointwise_regularizer=l2(0.0001)
    )(x)
    x = BatchNormalization()(x)
    
    # SE block
    if use_se:
        x = se_block(x)
    
    # Adjust shortcut if needed
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(
            filters, 1, strides=stride, padding='same',
            kernel_initializer='he_uniform',
            kernel_regularizer=l2(0.0001)
        )(shortcut)
        shortcut = BatchNormalization()(shortcut)
    
    # Add and activate
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    
    return x


: 

In [ ]:
def build_cnn_model(input_shape=(224, 224, 3), num_classes=3):
    """
    Build improved CNN with residual connections and SE blocks
    """
    inputs = Input(shape=input_shape)
    
    # Stem: Initial feature extraction
    x = Conv2D(32, 7, strides=2, padding='same',
               kernel_initializer='he_uniform',
               kernel_regularizer=l2(0.0001))(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(3, strides=2, padding='same')(x)
    
    # Stage 1: Low-level features
    x = residual_block(x, 64, stride=1, use_se=True)
    x = residual_block(x, 64, stride=1, use_se=True)
    x = Dropout(0.1)(x)
    
    # Stage 2: Mid-level features
    x = residual_block(x, 128, stride=2, use_se=True)
    x = residual_block(x, 128, stride=1, use_se=True)
    x = Dropout(0.15)(x)
    
    # Stage 3: High-level features
    x = residual_block(x, 256, stride=2, use_se=True)
    x = residual_block(x, 256, stride=1, use_se=True)
    x = Dropout(0.2)(x)
    
    # Stage 4: Deep features
    x = residual_block(x, 512, stride=2, use_se=True)
    x = residual_block(x, 512, stride=1, use_se=True)
    x = Dropout(0.25)(x)
    
    # Global pooling
    x = GlobalAveragePooling2D()(x)
    
    # Classification head
    x = Dense(256, kernel_initializer='he_uniform',
              kernel_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    
    # Output
    outputs = Dense(num_classes, activation='softmax',
                   kernel_regularizer=l2(0.0001))(x)
    
    model = Model(inputs=inputs, outputs=outputs, name='ImprovedCNN')
    
    return model

model_CNN = build_cnn_model()

: 

In [ ]:
model_CNN.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

: 

In [ ]:
model_CNN.summary()

: 

## Callbacks

In [ ]:
# Callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        'best_cnn_model_weighted.keras',
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=1
    ),
    
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        mode='min',
        verbose=1,
        min_delta=0.001
    ),
    
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=8,
        min_lr=1e-6,
        mode='min',
        verbose=1
    ),
    
    tf.keras.callbacks.LambdaCallback(
        on_epoch_end=lambda epoch, logs: print(
            f"\nEpoch {epoch+1} Summary:\n"
            f"   Train Loss: {logs['loss']:.4f} | Train Acc: {logs['accuracy']:.4f}\n"
            f"   Val Loss: {logs['val_loss']:.4f} | Val Acc: {logs['val_accuracy']:.4f}\n"
            f"   Gap: {abs(logs['accuracy'] - logs['val_accuracy']):.4f}"
        ) if epoch % 5 == 0 else None
    )
]

: 

## Data Augementation

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    zoom_range=0.15,
    shear_range=0.15,
    fill_mode='nearest'
)

: 

## show augmented samples

In [ ]:
def show_augmented_samples(X_sample, y_sample):
    fig, axes = plt.subplots(3, 5, figsize=(15, 9))
    
    img = X_sample[0:1]
    label = y_sample[0:1]
    
    axes[0, 0].imshow(img[0])
    axes[0, 0].set_title('Original')
    axes[0, 0].axis('off')
    
    for i in range(1, 15):
        row = i // 5
        col = i % 5
        
        # Generate augmented image
        aug_iter = train_datagen.flow(img, label, batch_size=1)
        aug_img = next(aug_iter)[0][0]
        
        axes[row, col].imshow(aug_img)
        axes[row, col].set_title(f'Aug {i}')
        axes[row, col].axis('off')
    
    plt.suptitle('Original vs Augmented Images')
    plt.tight_layout()
    plt.show()

show_augmented_samples(X_train, y_train)

: 

## fit cnn modle with class weights

In [ ]:
history = model_CNN.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=32, seed=42),
    validation_data=(X_test, y_test_cat),
    epochs=50,
    class_weight=class_weight_dict,
    steps_per_epoch=len(X_train) // 32,
    callbacks=callbacks,
    verbose=1
)

: 

In [ ]:
cnn_model = load_model('best_cnn_model_weighted.keras')

class_names = ['Early Blight', 'Late Blight', 'Healthy']

: 

## 1. TEST ACCURACY


In [ ]:
results = cnn_model.evaluate(X_test, y_test_cat, verbose=0)

if isinstance(results, list):
    test_loss = results[0]
    test_accuracy = results[1] if len(results) > 1 else results[0]
else:
    test_loss = results
    test_accuracy = results

print(f"\nTest Results:")
print(f"   • Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"   • Loss: {test_loss:.4f}")

: 

## 2. PREDICTIONS

In [ ]:
y_pred = cnn_model.predict(X_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test_cat, axis=1) if len(y_test_cat.shape) > 1 else y_test_cat

: 

## 3. CONFUSION MATRIX


In [ ]:
cm = confusion_matrix(y_true_classes, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names,
            yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.title('Potato Disease Classification - Confusion Matrix', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)

for i in range(len(class_names)):
    for j in range(len(class_names)):
        percentage = cm[i, j] / cm[i].sum() * 100
        plt.text(j + 0.5, i + 0.7, f'({percentage:.1f}%)', 
                ha='center', va='center', fontsize=9, color='gray')

plt.tight_layout()
plt.show()

: 

## 4. CLASSIFICATION REPORT


In [ ]:
print("\n" + "=" * 60)
print("DETAILED PERFORMANCE REPORT")
print("=" * 60)

report = classification_report(y_true_classes, y_pred_classes, 
                              target_names=class_names,
                              output_dict=True)

for class_name in class_names:
    print(f"\n {class_name}:")
    print(f"   • Precision: {report[class_name]['precision']:.3f}")
    print(f"   • Recall: {report[class_name]['recall']:.3f}")
    print(f"   • F1-Score: {report[class_name]['f1-score']:.3f}")
    print(f"   • Samples: {int(report[class_name]['support'])}")

print(f"\n Overall Performance:")
print(f"   • Accuracy: {report['accuracy']:.3f}")
print(f"   • Average F1-Score: {report['macro avg']['f1-score']:.3f}")

: 

## 5. PERFORMANCE VISUALIZATION


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Chart 1: Class-wise Metrics
ax1 = axes[0]
metrics = ['Precision', 'Recall', 'F1-Score']
x = np.arange(len(class_names))
width = 0.25

precision = [report[cn]['precision'] for cn in class_names]
recall = [report[cn]['recall'] for cn in class_names]
f1_score = [report[cn]['f1-score'] for cn in class_names]

ax1.bar(x - width, precision, width, label='Precision', color='#3498db')
ax1.bar(x, recall, width, label='Recall', color='#2ecc71')
ax1.bar(x + width, f1_score, width, label='F1-Score', color='#e74c3c')

ax1.set_xlabel('Disease Type')
ax1.set_ylabel('Score')
ax1.set_title('Performance Metrics by Disease Type')
ax1.set_xticks(x)
ax1.set_xticklabels(class_names, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Chart 2: Sample Distribution
ax2 = axes[1]
support = [report[cn]['support'] for cn in class_names]
colors = ['#ff9999', '#66b3ff', '#99ff99']
wedges, texts, autotexts = ax2.pie(support, labels=class_names, autopct='%1.1f%%',
                                    colors=colors, startangle=90)
ax2.set_title('Test Set Distribution')

# Chart 3: Accuracy per Class
ax3 = axes[2]
class_accuracy = []
for i in range(len(class_names)):
    if cm[i].sum() > 0:
        acc = cm[i, i] / cm[i].sum()
        class_accuracy.append(acc)
    else:
        class_accuracy.append(0)

bars = ax3.bar(class_names, class_accuracy, color=['#ff6b6b', '#4ecdc4', '#45b7d1'])
ax3.set_title('Accuracy per Disease Type')
ax3.set_ylabel('Accuracy')
ax3.set_ylim([0, 1.1])
ax3.grid(True, alpha=0.3)

# Add percentage labels
for bar, acc in zip(bars, class_accuracy):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{acc:.1%}', ha='center', va='bottom', fontweight='bold')

plt.suptitle('Potato Disease Detection - Performance Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()


: 

## 6. ERROR ANALYSIS


In [ ]:
print("\n" + "=" * 60)
print("ERROR ANALYSIS")
print("=" * 60)

total_samples = len(y_true_classes)
correct_predictions = np.sum(y_pred_classes == y_true_classes)
wrong_predictions = total_samples - correct_predictions

print(f"\n Correct Predictions: {correct_predictions}/{total_samples} ({correct_predictions/total_samples*100:.1f}%)")
print(f"Wrong Predictions: {wrong_predictions}/{total_samples} ({wrong_predictions/total_samples*100:.1f}%)")

print("\nCommon Misclassifications:")
for i, true_class in enumerate(class_names):
    for j, pred_class in enumerate(class_names):
        if i != j and cm[i, j] > 0:
            print(f"   • {true_class} → {pred_class}: {cm[i, j]} times")

: 

# 7. FINAL SUMMARY

In [ ]:
print("\n" + "=" * 60)
print("FINAL SUMMARY")
print("=" * 60)

f1_scores = [report[cn]['f1-score'] for cn in class_names]
best_class = class_names[np.argmax(f1_scores)]
worst_class = class_names[np.argmin(f1_scores)]

print(f"\n Potato Disease Detection Model:")
print(f"   • Overall Accuracy: {test_accuracy*100:.2f}%")
print(f"   • Best Detection: {best_class} (F1: {max(f1_scores):.3f})")
print(f"   • Needs Improvement: {worst_class} (F1: {min(f1_scores):.3f})")


print("\n" + "=" * 60)
print("Evaluation Complete!")
print("=" * 60)

import json

results_summary = {
    'model': 'Potato Disease Classification',
    'categories': categories,
    'test_accuracy': float(test_accuracy),
    'test_loss': float(test_loss),
    'per_class_accuracy': {cn: float(acc) for cn, acc in zip(class_names, class_accuracy)},
    'f1_scores': {cn: float(f1) for cn, f1 in zip(class_names, f1_scores)},
    'total_samples': int(total_samples),
    'correct_predictions': int(correct_predictions)
}

with open('potato_model_results.json', 'w') as f:
    json.dump(results_summary, f, indent=4)

print("\nResults saved to 'potato_model_results.json'")

: 

## Transfer Learning with InceptionV3

In [ ]:
tf.keras.backend.clear_session()

: 

In [ ]:
from tensorflow.keras.applications import InceptionV3
base_model = InceptionV3(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'   # use pretrained on imagenet
)

base_model.trainable = False

: 

## Add custom layers


In [ ]:
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
outputs = Dense(3, activation='softmax')(x)

: 

## Create the model

In [ ]:
model_inception = Model(inputs, outputs)

: 

## Compile with low learning rate


In [ ]:
print(f"Total layers: {len(model_inception.layers)}")
print(f"Trainable parameters: {model_inception.count_params():,}")

: 

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-100]:
    layer.trainable = False

: 

In [ ]:
model_inception.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

: 

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        'best_inceptionV3.keras',
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=1
    ),
    
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        mode='min',
        verbose=1,
        min_delta=0.001
    ),
    
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=8,
        min_lr=1e-6,
        mode='min',
        verbose=1
    ),
    
    tf.keras.callbacks.LambdaCallback(
        on_epoch_end=lambda epoch, logs: print(
            f"\nEpoch {epoch+1} Summary:\n"
            f"   Train Loss: {logs['loss']:.4f} | Train Acc: {logs['accuracy']:.4f}\n"
            f"   Val Loss: {logs['val_loss']:.4f} | Val Acc: {logs['val_accuracy']:.4f}\n"
            f"   Gap: {abs(logs['accuracy'] - logs['val_accuracy']):.4f}"
        ) if epoch % 5 == 0 else None
    )
]

: 

In [ ]:
model_inception_history = model_inception.fit(
    train_datagen.flow(X_train, y_train_cat, batch_size=32),
    validation_data=(X_test, y_test_cat),
    epochs=40,
    class_weight=class_weight_dict,
    steps_per_epoch=len(X_train) // 32,
    callbacks=callbacks,
    verbose=1
)

: 

## CONFUSION MATRIX 

In [ ]:
plt.figure(figsize=(10, 8))

inception_model = load_model('best_inceptionV3.keras')

y_pred = inception_model.predict(X_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test_cat, axis=1) if len(y_test_cat.shape) > 1 else y_test_cat

cm = confusion_matrix(y_true_classes, y_pred_classes)

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            square=True, linewidths=1, linecolor='black',
            xticklabels=class_names,
            yticklabels=class_names,
            cbar_kws={'label': 'Count'})

plt.title('Potato Disease Classification - Confusion Matrix', 
          fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Actual Disease', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Disease', fontsize=14, fontweight='bold')

for i in range(len(class_names)):
    for j in range(len(class_names)):
        percentage = (cm[i, j] / cm[i].sum()) * 100
        plt.text(j + 0.5, i + 0.7, f'({percentage:.1f}%)', 
                ha='center', va='center', fontsize=10, color='darkred')

plt.tight_layout()
plt.show()

: 

## PRINT RESULTS


In [ ]:
print("=" * 60)
print("CONFUSION MATRIX RESULTS")
print("=" * 60)

print("\nAccuracy per Disease:")
print("-" * 40)
for i, disease in enumerate(class_names):
    total = cm[i].sum()
    correct = cm[i, i]
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"{disease:15} → {correct}/{total} correct ({accuracy:.1f}%)")

print("\nCommon Mistakes:")
print("-" * 40)
for i in range(len(class_names)):
    for j in range(len(class_names)):
        if i != j and cm[i, j] > 0:
            percentage = (cm[i, j] / cm[i].sum()) * 100
            print(f"• {class_names[i]} misclassified as {class_names[j]}: {cm[i, j]} times ({percentage:.1f}%)")

total_correct = np.diag(cm).sum()
total_samples = cm.sum()
overall_accuracy = (total_correct / total_samples) * 100

print("\n" + "=" * 60)
print(f"Overall Accuracy: {total_correct}/{total_samples} ({overall_accuracy:.2f}%)")
print("=" * 60)

: 

# Model Performance

## 1. ACCURACY & LOSS CURVES

In [ ]:
print("=" * 60)
print(" INCEPTION MODEL - TRAINING RESULTS")
print("=" * 60)

history = model_inception_history.history
epochs = range(1, len(history['loss']) + 1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy Plot
ax1.plot(epochs, history['accuracy'], 'b-', label='Training Accuracy', linewidth=2)
ax1.plot(epochs, history['val_accuracy'], 'r-', label='Validation Accuracy', linewidth=2)
ax1.fill_between(epochs, history['accuracy'], alpha=0.1, color='blue')
ax1.fill_between(epochs, history['val_accuracy'], alpha=0.1, color='red')
ax1.set_title(' Model Accuracy', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 1.05])

# Add best accuracy point
best_val_acc = max(history['val_accuracy'])
best_epoch = history['val_accuracy'].index(best_val_acc) + 1
ax1.plot(best_epoch, best_val_acc, 'g*', markersize=15, label=f'Best: {best_val_acc:.3f}')
ax1.legend(loc='lower right')

# Loss Plot
ax2.plot(epochs, history['loss'], 'b-', label='Training Loss', linewidth=2)
ax2.plot(epochs, history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
ax2.fill_between(epochs, history['loss'], alpha=0.1, color='blue')
ax2.fill_between(epochs, history['val_loss'], alpha=0.1, color='red')
ax2.set_title(' Model Loss', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

# Add best loss point
best_val_loss = min(history['val_loss'])
best_loss_epoch = history['val_loss'].index(best_val_loss) + 1
ax2.plot(best_loss_epoch, best_val_loss, 'g*', markersize=15, label=f'Best: {best_val_loss:.3f}')
ax2.legend(loc='upper right')

plt.suptitle('Potato Disease Detection - InceptionV3 Training History', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

: 

## 2. TRAINING SUMMARY


In [ ]:
print("\n Training Summary:")
print("-" * 40)
print(f"Total Epochs: {len(epochs)}")
print(f"Best Validation Accuracy: {best_val_acc:.4f} (Epoch {best_epoch})")
print(f"Best Validation Loss: {best_val_loss:.4f} (Epoch {best_loss_epoch})")
print(f"Final Training Accuracy: {history['accuracy'][-1]:.4f}")
print(f"Final Validation Accuracy: {history['val_accuracy'][-1]:.4f}")
print(f"Final Training Loss: {history['loss'][-1]:.4f}")
print(f"Final Validation Loss: {history['val_loss'][-1]:.4f}")

: 

## 3. OVERFITTING CHECK


In [ ]:
print("\nModel Analysis:")
print("-" * 40)

# Check for overfitting
final_train_acc = history['accuracy'][-1]
final_val_acc = history['val_accuracy'][-1]
acc_diff = final_train_acc - final_val_acc
print(f"Gap between training and validation: {acc_diff:.3f}")

: 

## 4. IMPROVEMENT ANALYSIS


In [ ]:
# Calculate improvement
initial_acc = history['val_accuracy'][0]
improvement = best_val_acc - initial_acc
improvement_percentage = (improvement / initial_acc) * 100

print(f"\nPerformance Improvement:")
print(f"   • Initial Accuracy: {initial_acc:.4f}")
print(f"   • Best Accuracy: {best_val_acc:.4f}")
print(f"   • Improvement: {improvement:.4f} ({improvement_percentage:.1f}%)")

: 

## 5. SIMPLE METRICS BAR CHART


In [ ]:
plt.figure(figsize=(10, 6))

metrics = ['Initial\nAccuracy', 'Final\nAccuracy', 'Best\nAccuracy']
train_values = [history['accuracy'][0], history['accuracy'][-1], max(history['accuracy'])]
val_values = [history['val_accuracy'][0], history['val_accuracy'][-1], best_val_acc]

x = np.arange(len(metrics))
width = 0.35

bars1 = plt.bar(x - width/2, train_values, width, label='Training', color='#3498db')
bars2 = plt.bar(x + width/2, val_values, width, label='Validation', color='#e74c3c')

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontweight='bold')

plt.xlabel('Metrics', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Model Performance Comparison', fontsize=14, fontweight='bold')
plt.xticks(x, metrics)
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.ylim([0, 1.1])

plt.tight_layout()
plt.show()


: 

## 6. SAVE TRAINING HISTORY

In [ ]:

training_summary = {
    'model': 'InceptionV3 - Potato Disease Detection',
    'total_epochs': len(epochs),
    'best_validation_accuracy': float(best_val_acc),
    'best_epoch': int(best_epoch),
    'final_training_accuracy': float(history['accuracy'][-1]),
    'final_validation_accuracy': float(history['val_accuracy'][-1]),
    'improvement': float(improvement),
    'improvement_percentage': float(improvement_percentage)
}

with open('inception_training_history.json', 'w') as f:
    json.dump(training_summary, f, indent=4)

print("\nTraining history saved to 'inception_training_history.json'")

: 

In [ ]:
inception_model.save_weights('inception_weights.weights.h5')
cnn_model.save_weights('cnn_weights.weights.h5')

: 